In [434]:
# Dependencies
from config import gkey
import pytz
from datetime import datetime
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from pprint import pprint
from geopy.geocoders import Nominatim
from geopy import geocoders 
from scipy.stats import linregress
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import os
import json
import urllib.request

# Google developer API key
from config import gkey

In [359]:
accident_data_to_load = "Resources/US_Accidents_May19.csv"
accidents_data_df = pd.read_csv(
    accident_data_to_load)

# Visualize
accidents_data_df = accidents_data_df.reset_index()
accidents_data_df.set_index('ID', inplace=True)
accidents_data_df.head()


,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,29.68,10.0,Calm,NaN,0.02,Light Rain,False,Night,Night,Night
A-2,1,OH,2,2/8/16 6:07,2/8/16 6:37,39.928059,-82.831184,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,...,29.65,10.0,Calm,NaN,0.00,Light Rain,False,Night,Night,Day
A-3,2,OH,2,2/8/16 6:49,2/8/16 7:19,39.063148,-84.032608,0.01,Accident on OH-32 State Route 32 Westbound at ...,NaN,...,29.67,10.0,SW,3.5,NaN,Overcast,True,Night,Night,Day
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day
A-5,4,OH,2,2/8/16 7:39,2/8/16 8:09,39.627781,-84.188354,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,...,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,True,Day,Day,Day


In [360]:
accidents_data_df['Street']

ID
A-1                             I-70 E
A-2                           Brice Rd
A-3                     State Route 32
A-4                             I-75 S
A-5          Miamisburg Centerville Rd
                       ...            
A-1048571               Piney Grove Rd
A-1048572                    Farrow Rd
A-1048573              Moreland Ave SE
A-1048574                 Two Notch Rd
A-1048575               Tidal Point Ln
Name: Street, Length: 1048575, dtype: object

In [361]:
accidents_data_hwy_df = accidents_data_df['Street'].str.split("-", n = 1, expand = True)[1].str.contains("E|W")== True
accidents_data_hwy_df.value_counts()


False    924716
True     123859
Name: 1, dtype: int64

In [362]:
accidents_data_hwy_df.head()

ID
A-1     True
A-2    False
A-3    False
A-4    False
A-5    False
Name: 1, dtype: bool

In [363]:
accidents_data_df['hwy'] = accidents_data_df['Street'].str.split("-", n = 1, expand = True)[1].str.contains("E|W")== True
accidents_data_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,10.0,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True
A-2,1,OH,2,2/8/16 6:07,2/8/16 6:37,39.928059,-82.831184,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,...,10.0,Calm,NaN,0.00,Light Rain,False,Night,Night,Day,False
A-3,2,OH,2,2/8/16 6:49,2/8/16 7:19,39.063148,-84.032608,0.01,Accident on OH-32 State Route 32 Westbound at ...,NaN,...,10.0,SW,3.5,NaN,Overcast,True,Night,Night,Day,False
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,9.0,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,False
A-5,4,OH,2,2/8/16 7:39,2/8/16 8:09,39.627781,-84.188354,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,...,6.0,SW,3.5,NaN,Mostly Cloudy,True,Day,Day,Day,False


In [364]:
accidents_data_hwy_df = accidents_data_df.loc[accidents_data_df['hwy'] == True]
accidents_data_hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,10.0,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,3.0,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True
A-59,58,OH,2,2/9/16 7:42,2/9/16 8:12,39.754520,-84.159279,0.01,Accident on US-35 Eastbound at Steve Whalen Bl...,NaN,...,1.2,SW,9.2,0.00,Light Snow,False,Day,Day,Day,True
A-60,59,OH,2,2/9/16 7:42,2/9/16 8:12,39.749916,-84.139359,0.01,Accident on US-35 Eastbound at Smithville Rd. ...,NaN,...,1.2,SW,9.2,0.00,Light Snow,False,Day,Day,Day,True
A-69,68,OH,3,2/9/16 8:35,2/9/16 9:05,41.424404,-81.578674,0.01,Accident on I-480 Eastbound at Exit 23 OH-14 B...,NaN,...,10.0,WSW,12.7,NaN,Overcast,False,Day,Day,Day,True


In [365]:
# Output File (CSV)
os.makedirs(os.path.join('output_data'),exist_ok=True)
output_data_file = os.path.join("output_data","accidents_data_hwy_df.csv")
accidents_data_hwy_df.to_csv(output_data_file)

In [366]:
accident_data_hwy_to_load = "Resources/accidents_data_hwy_df.csv"
accidents_hwy_west_df = pd.read_csv(
    accident_data_hwy_to_load, low_memory=False)

# Visualize
accidents_hwy_west_df = accidents_hwy_west_df.reset_index()
accidents_hwy_west_df.set_index('ID', inplace=True)
accidents_hwy_west_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,...,10.0,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,3.0,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True
A-59,2,58,OH,2,2/9/16 7:42,2/9/16 8:12,39.754520,-84.159279,0.01,Accident on US-35 Eastbound at Steve Whalen Bl...,...,1.2,SW,9.2,0.00,Light Snow,False,Day,Day,Day,True
A-60,3,59,OH,2,2/9/16 7:42,2/9/16 8:12,39.749916,-84.139359,0.01,Accident on US-35 Eastbound at Smithville Rd. ...,...,1.2,SW,9.2,0.00,Light Snow,False,Day,Day,Day,True
A-69,4,68,OH,3,2/9/16 8:35,2/9/16 9:05,41.424404,-81.578674,0.01,Accident on I-480 Eastbound at Exit 23 OH-14 B...,...,10.0,WSW,12.7,NaN,Overcast,False,Day,Day,Day,True


In [367]:
accidents_hwy_west_df['Direction_West'] = accidents_hwy_west_df['Description'].str.contains("Westbound")== True
accidents_hwy_west_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,...,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True,False
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True,True
A-59,2,58,OH,2,2/9/16 7:42,2/9/16 8:12,39.754520,-84.159279,0.01,Accident on US-35 Eastbound at Steve Whalen Bl...,...,SW,9.2,0.00,Light Snow,False,Day,Day,Day,True,False
A-60,3,59,OH,2,2/9/16 7:42,2/9/16 8:12,39.749916,-84.139359,0.01,Accident on US-35 Eastbound at Smithville Rd. ...,...,SW,9.2,0.00,Light Snow,False,Day,Day,Day,True,False
A-69,4,68,OH,3,2/9/16 8:35,2/9/16 9:05,41.424404,-81.578674,0.01,Accident on I-480 Eastbound at Exit 23 OH-14 B...,...,WSW,12.7,NaN,Overcast,False,Day,Day,Day,True,False


In [368]:
accidents_data_hwy_westbound_df = accidents_hwy_west_df.loc[accidents_hwy_west_df['Direction_West'] == True]
accidents_data_hwy_westbound_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True,True
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,WNW,10.4,0.00,Light Snow,False,Day,Day,Day,True,True
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,SSW,4.6,NaN,Scattered Clouds,False,Day,Day,Day,True,True
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,SE,5.8,0.00,Light Freezing Drizzle,False,Night,Night,Night,True,True
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,SSE,8.1,NaN,Light Snow,False,Night,Night,Day,True,True


In [369]:
len(accidents_data_hwy_westbound_df)


52726

In [370]:
accidents_data_hwy_westbound_df['Accident_Date'] = accidents_data_hwy_westbound_df['Start_Time'].str.split(" ", n = 1, expand = True)[0]
accidents_data_hwy_westbound_df.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Accident_Date
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,4.6,0.02,Light Rain,False,Day,Day,Day,True,True,2/8/16
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,10.4,0.00,Light Snow,False,Day,Day,Day,True,True,2/10/16
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,4.6,NaN,Scattered Clouds,False,Day,Day,Day,True,True,2/11/16
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,5.8,0.00,Light Freezing Drizzle,False,Night,Night,Night,True,True,2/15/16
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,8.1,NaN,Light Snow,False,Night,Night,Day,True,True,2/15/16


In [371]:
accidents_data_hwy_westbound_df['Accident_Date'] = pd.to_datetime(accidents_data_hwy_westbound_df['Accident_Date'])
accidents_data_hwy_westbound_df['Accident_Date_No_Time'] = accidents_data_hwy_westbound_df['Accident_Date'].dt.date
accidents_data_hwy_westbound_df.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Accident_Date,Accident_Date_No_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,0.02,Light Rain,False,Day,Day,Day,True,True,2016-02-08,2016-02-08
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,0.00,Light Snow,False,Day,Day,Day,True,True,2016-02-10,2016-02-10
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,NaN,Scattered Clouds,False,Day,Day,Day,True,True,2016-02-11,2016-02-11
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,0.00,Light Freezing Drizzle,False,Night,Night,Night,True,True,2016-02-15,2016-02-15
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,NaN,Light Snow,False,Night,Night,Day,True,True,2016-02-15,2016-02-15


In [372]:
"""
request format :
https://api.sunrise-sunset.org/json?lat=36.7201600&lng=-4.4203400&date=2019-12-26

"""
base_url = "https://api.sunrise-sunset.org/json?"
latitude = []
longitude = []
date_accident = []
sunset_time = []

for city, state, zipcode, date in zip(accidents_data_hwy_westbound_df.loc[:, 'City'], accidents_data_hwy_westbound_df.loc[:, 'State'], accidents_data_hwy_westbound_df.loc[:, 'Zipcode'], accidents_data_hwy_westbound_df.loc[:, 'Accident_Date_No_Time']):
    
    try:
        #print(city, state, zipcode, date)
        # Target city
        target_city = f"{city}, {state}"

        # Build the endpoint URL
        target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
        'address={0}&key={1}').format(target_city, gkey)
        #Run a request to endpoint and convert result to json
        geo_data = requests.get(target_url).json()

        # Extract latitude and longitude
        lat = geo_data["results"][0]["geometry"]["location"]["lat"]
        lng = geo_data["results"][0]["geometry"]["location"]["lng"]

        # Print the latitude and longitude
        print('''
             City: {0}
             Latitude: {1}
             Longitude: {2}
              '''.format(target_city, lat, lng))


        sunset_url = f'{base_url}lat={lat}&lng={lng}&date={date}'
        #print(sunset_url)
        response = requests.get(sunset_url, verify=False)
        sunset_data = response.json()
        #pprint(sunset_data)
        print('Sunset:', sunset_data['results']['sunset'])
        print("\n")

        latitude.append(lat)
        longitude.append(lng)
        date_accident.append(date)
        sunset_time.append(sunset_data['results']['sunset'])

    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
        print("Connection refused by the server..")
        print("Let me sleep for 5 seconds")
        time.sleep(5)
        print("Was a nice sleep, now let me continue...")
        continue
    
    
    


             City: Reynoldsburg, OH
             Latitude: 39.9547861
             Longitude: -82.81211909999999
              
Sunset: 10:58:48 PM




/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)



             City: Barberton, OH
             Latitude: 41.012833
             Longitude: -81.6051221
              
Sunset: 10:54:32 PM




/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)



             City: Dayton, OH
             Latitude: 39.7589478
             Longitude: -84.1916069
              
Sunset: 11:08:13 PM




...

/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Sunset: 3:23:43 AM



             City: Oakland, CA
             Latitude: 37.8043514
             Longitude: -122.2711639
              


/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Sunset: 3:26:04 AM




AttributeError: 'dict' object has no attribute 'status_code'

In [373]:
#print(sunset_time)
len(sunset_time)

3761

In [374]:
accidents_data_hwy_westbound_sunset_df = accidents_data_hwy_westbound_df.iloc[:len(sunset_time), :]
accidents_data_hwy_westbound_sunset_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Accident_Date,Accident_Date_No_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,0.02,Light Rain,False,Day,Day,Day,True,True,2016-02-08,2016-02-08
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,0.00,Light Snow,False,Day,Day,Day,True,True,2016-02-10,2016-02-10
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,NaN,Scattered Clouds,False,Day,Day,Day,True,True,2016-02-11,2016-02-11
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,0.00,Light Freezing Drizzle,False,Night,Night,Night,True,True,2016-02-15,2016-02-15
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,NaN,Light Snow,False,Night,Night,Day,True,True,2016-02-15,2016-02-15


In [375]:
len(accidents_data_hwy_westbound_sunset_df)

3761

In [376]:
accidents_data_hwy_westbound_sunset_df['Sunset_Time_UTC'] = sunset_time
accidents_data_hwy_westbound_sunset_df.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Accident_Date,Accident_Date_No_Time,Sunset_Time_UTC
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,Light Rain,False,Day,Day,Day,True,True,2016-02-08,2016-02-08,10:58:48 PM
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,Light Snow,False,Day,Day,Day,True,True,2016-02-10,2016-02-10,10:54:32 PM
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,Scattered Clouds,False,Day,Day,Day,True,True,2016-02-11,2016-02-11,11:08:13 PM
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,Light Freezing Drizzle,False,Night,Night,Night,True,True,2016-02-15,2016-02-15,11:07:50 PM
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,Light Snow,False,Night,Night,Day,True,True,2016-02-15,2016-02-15,11:07:50 PM


In [377]:
del accidents_data_hwy_westbound_sunset_df['Accident_Date']
accidents_data_hwy_westbound_sunset_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Accident_Date_No_Time,Sunset_Time_UTC
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,0.02,Light Rain,False,Day,Day,Day,True,True,2016-02-08,10:58:48 PM
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,0.00,Light Snow,False,Day,Day,Day,True,True,2016-02-10,10:54:32 PM
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,NaN,Scattered Clouds,False,Day,Day,Day,True,True,2016-02-11,11:08:13 PM
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,0.00,Light Freezing Drizzle,False,Night,Night,Night,True,True,2016-02-15,11:07:50 PM
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,NaN,Light Snow,False,Night,Night,Day,True,True,2016-02-15,11:07:50 PM


In [378]:
accidents_data_hwy_westbound_sunset_df = accidents_data_hwy_westbound_sunset_df.rename({'Accident_Date_No_Time':'Accident_Date'}, axis=1)
accidents_data_hwy_westbound_sunset_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Accident_Date,Sunset_Time_UTC
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,0.02,Light Rain,False,Day,Day,Day,True,True,2016-02-08,10:58:48 PM
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,0.00,Light Snow,False,Day,Day,Day,True,True,2016-02-10,10:54:32 PM
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,NaN,Scattered Clouds,False,Day,Day,Day,True,True,2016-02-11,11:08:13 PM
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,0.00,Light Freezing Drizzle,False,Night,Night,Night,True,True,2016-02-15,11:07:50 PM
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,NaN,Light Snow,False,Night,Night,Day,True,True,2016-02-15,11:07:50 PM


In [379]:
accidents_data_hwy_westbound_sunset_df['Accident_Time'] = accidents_data_hwy_westbound_sunset_df['Start_Time'].str.split(" ", n = 1, expand = True)[1]
accidents_data_hwy_westbound_sunset_df['Accident_Date'] = accidents_data_hwy_westbound_sunset_df['Start_Time'].str.split(" ", n = 1, expand = True)[0]
accidents_data_hwy_westbound_sunset_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Accident_Date,Sunset_Time_UTC,Accident_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,Light Rain,False,Day,Day,Day,True,True,2/8/16,10:58:48 PM,8:21
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,Light Snow,False,Day,Day,Day,True,True,2/10/16,10:54:32 PM,17:10
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,Scattered Clouds,False,Day,Day,Day,True,True,2/11/16,11:08:13 PM,8:13
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,Light Freezing Drizzle,False,Night,Night,Night,True,True,2/15/16,11:07:50 PM,5:42
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,Light Snow,False,Night,Night,Day,True,True,2/15/16,11:07:50 PM,6:36


In [380]:
accidents_data_hwy_westbound_sunset_df['Accident_Date'] = accidents_data_hwy_westbound_sunset_df['Accident_Date'].str.replace('/','-')
accidents_data_hwy_westbound_sunset_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Accident_Date,Sunset_Time_UTC,Accident_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,Light Rain,False,Day,Day,Day,True,True,2-8-16,10:58:48 PM,8:21
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,Light Snow,False,Day,Day,Day,True,True,2-10-16,10:54:32 PM,17:10
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,Scattered Clouds,False,Day,Day,Day,True,True,2-11-16,11:08:13 PM,8:13
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,Light Freezing Drizzle,False,Night,Night,Night,True,True,2-15-16,11:07:50 PM,5:42
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,Light Snow,False,Night,Night,Day,True,True,2-15-16,11:07:50 PM,6:36


In [381]:
# create timezones based on states, since timezone column is provided will comment this code for now
"""
conditions = [
    (accidents_data_hwy_westbound_sunset_df['State'] == 'OH'),
    (accidents_data_hwy_westbound_sunset_df['State'] == 'CA'),
]
choices = ['US/Eastern', 'US/Pacific']
accidents_data_hwy_westbound_sunset_df['Timezone'] = np.select(conditions, choices, default='US/Eastern')
accidents_data_hwy_westbound_sunset_df.head()

"""

"\nconditions = [\n    (accidents_data_hwy_westbound_sunset_df['State'] == 'OH'),\n    (accidents_data_hwy_westbound_sunset_df['State'] == 'CA'),\n]\nchoices = ['US/Eastern', 'US/Pacific']\naccidents_data_hwy_westbound_sunset_df['Timezone'] = np.select(conditions, choices, default='US/Eastern')\naccidents_data_hwy_westbound_sunset_df.head()\n\n"

In [382]:
accidents_data_hwy_westbound_sunset_df['Sunset_Time_UTC_Hours'] = pd.to_datetime(accidents_data_hwy_westbound_sunset_df['Sunset_Time_UTC']).dt.strftime('%H:%M:%S')
accidents_data_hwy_westbound_sunset_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Time_UTC_Hours
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,False,Day,Day,Day,True,True,2-8-16,10:58:48 PM,8:21,22:58:48
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,False,Day,Day,Day,True,True,2-10-16,10:54:32 PM,17:10,22:54:32
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,False,Day,Day,Day,True,True,2-11-16,11:08:13 PM,8:13,23:08:13
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,False,Night,Night,Night,True,True,2-15-16,11:07:50 PM,5:42,23:07:50
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,False,Night,Night,Day,True,True,2-15-16,11:07:50 PM,6:36,23:07:50


In [383]:
accidents_data_hwy_westbound_sunset_date_time_df = accidents_data_hwy_westbound_sunset_df.assign(Sunset_Date_Time = accidents_data_hwy_westbound_sunset_df.Accident_Date.astype(str) + ' ' + \
  accidents_data_hwy_westbound_sunset_df.Sunset_Time_UTC_Hours.astype(str) )
accidents_data_hwy_westbound_sunset_date_time_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Time_UTC_Hours,Sunset_Date_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,Day,Day,Day,True,True,2-8-16,10:58:48 PM,8:21,22:58:48,2-8-16 22:58:48
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,Day,Day,Day,True,True,2-10-16,10:54:32 PM,17:10,22:54:32,2-10-16 22:54:32
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,Day,Day,Day,True,True,2-11-16,11:08:13 PM,8:13,23:08:13,2-11-16 23:08:13
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,Night,Night,Night,True,True,2-15-16,11:07:50 PM,5:42,23:07:50,2-15-16 23:07:50
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,Night,Night,Day,True,True,2-15-16,11:07:50 PM,6:36,23:07:50,2-15-16 23:07:50


In [384]:
pytz.all_timezones

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Asmera',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Timbuktu',
 'Africa/

In [385]:
sunset_time_local = []

for date, time, timezone in zip(accidents_data_hwy_westbound_sunset_date_time_df.loc[:, 'Accident_Date'], accidents_data_hwy_westbound_sunset_date_time_df.loc[:, 'Sunset_Time_UTC_Hours'], accidents_data_hwy_westbound_sunset_date_time_df.loc[:, 'Timezone']):
    #print(date, time, timezone)  
    est = pytz.timezone('US/Eastern')
    pst = pytz.timezone('US/Pacific')
    mst = pytz.timezone('US/Mountain')
    cst = pytz.timezone('US/Central')
    ast = pytz.timezone('US/Alaska')
    hst = pytz.timezone('US/Hawaii')
    arst = pytz.timezone('US/Arizona')
    #utc = pytz.utc
    fmt = '%Y-%m-%d %H:%M:%S'

    utc_time = datetime(int('20'+ str(date).split('-')[2]), int(str(date).split('-')[0]), int(str(date).split('-')[1]), int(str(time).split(':')[0]), int(str(time).split(':')[1]), int(str(time).split(':')[2]), tzinfo=utc)
    
    if (timezone == 'US/Eastern'):    
        sunset_time_local.append(utc_time.astimezone(est).strftime(fmt))
    elif (timezone == 'US/Pacific'):        
        sunset_time_local.append(utc_time.astimezone(pst).strftime(fmt))
    elif (timezone == 'US/Mountain'):        
        sunset_time_local.append(utc_time.astimezone(mst).strftime(fmt))
    elif (timezone == 'US/Central'):        
        sunset_time_local.append(utc_time.astimezone(cst).strftime(fmt))
    elif (timezone == 'US/Alaska'):      
        sunset_time_local.append(utc_time.astimezone(ast).strftime(fmt))
    elif (timezone == 'US/Hawaii'):        
        sunset_time_local.append(utc_time.astimezone(hst).strftime(fmt))
    elif (timezone == 'US/Arizona'):        
        sunset_time_local.append(utc_time.astimezone(arst).strftime(fmt))
print(sunset_time_local)

['2016-02-08 17:58:48', '2016-02-10 17:54:32', '2016-02-11 18:08:13', '2016-02-15 18:07:50', '2016-02-15 18:07:50', '2016-02-16 18:09:00', '2016-02-17 18:06:15', '2016-02-26 18:13:49', '2016-03-03 18:31:57', '2016-03-04 18:28:07', '2016-03-10 18:34:32', '2016-03-15 19:37:55', '2016-03-16 19:40:23', '2016-06-20 20:32:42', '2016-06-20 20:34:34', '2016-06-20 20:34:17', '2016-06-20 20:34:00', '2016-06-20 20:32:23', '2016-06-20 20:33:50', '2016-06-20 20:34:43', '2016-06-20 20:31:08', '2016-06-20 20:34:43', '2016-06-20 20:35:16', '2016-06-20 20:31:08', '2016-06-21 20:33:42', '2016-06-21 20:33:10', '2016-06-21 20:34:10', '2016-06-21 20:35:35', '2016-06-21 20:35:36', '2016-06-21 20:31:18', '2016-06-21 20:35:36', '2016-06-21 20:34:10', '2016-06-21 20:33:29', '2016-06-21 20:35:03', '2016-06-21 20:34:10', '2016-06-21 20:35:19', '2016-06-22 20:35:27', '2016-06-22 20:34:18', '2016-06-22 20:35:10', '2016-06-22 20:35:27', '2016-06-22 20:32:04', '2016-06-22 20:35:01', '2016-06-22 20:32:38', '2016-06-2

In [386]:
accidents_data_hwy_westbound_sunset_date_time_df['Sunset_Date_Time_Local'] = sunset_time_local
accidents_data_hwy_westbound_sunset_date_time_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Civil_Twilight,Astronomical_Twilight,hwy,Direction_West,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Time_UTC_Hours,Sunset_Date_Time,Sunset_Date_Time_Local
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,Day,Day,True,True,2-8-16,10:58:48 PM,8:21,22:58:48,2-8-16 22:58:48,2016-02-08 17:58:48
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,Day,Day,True,True,2-10-16,10:54:32 PM,17:10,22:54:32,2-10-16 22:54:32,2016-02-10 17:54:32
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,Day,Day,True,True,2-11-16,11:08:13 PM,8:13,23:08:13,2-11-16 23:08:13,2016-02-11 18:08:13
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,Night,Night,True,True,2-15-16,11:07:50 PM,5:42,23:07:50,2-15-16 23:07:50,2016-02-15 18:07:50
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,Night,Day,True,True,2-15-16,11:07:50 PM,6:36,23:07:50,2-15-16 23:07:50,2016-02-15 18:07:50


In [387]:
accidents_data_hwy_westbound_sunset_date_time_df['Sunset_Time_Local'] = accidents_data_hwy_westbound_sunset_date_time_df['Sunset_Date_Time_Local'].str.split(' ', n = 1, expand = True)[1]
accidents_data_hwy_westbound_sunset_date_time_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Astronomical_Twilight,hwy,Direction_West,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Time_UTC_Hours,Sunset_Date_Time,Sunset_Date_Time_Local,Sunset_Time_Local
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,Day,True,True,2-8-16,10:58:48 PM,8:21,22:58:48,2-8-16 22:58:48,2016-02-08 17:58:48,17:58:48
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,Day,True,True,2-10-16,10:54:32 PM,17:10,22:54:32,2-10-16 22:54:32,2016-02-10 17:54:32,17:54:32
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,Day,True,True,2-11-16,11:08:13 PM,8:13,23:08:13,2-11-16 23:08:13,2016-02-11 18:08:13,18:08:13
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,Night,True,True,2-15-16,11:07:50 PM,5:42,23:07:50,2-15-16 23:07:50,2016-02-15 18:07:50,18:07:50
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,Day,True,True,2-15-16,11:07:50 PM,6:36,23:07:50,2-15-16 23:07:50,2016-02-15 18:07:50,18:07:50


In [388]:
time_range_status = []

def time_in_range(startTime, endTime, accidentTime):
    if startTime < endTime:
        return startTime <= accidentTime and accidentTime <= endTime
    else: #Over midnight
        return accidentTime >= startTime or accidentTime <= endTime

for accident_time, sunset_time_local in zip(accidents_data_hwy_westbound_sunset_date_time_df.loc[:, 'Accident_Time'], accidents_data_hwy_westbound_sunset_date_time_df.loc[:, 'Sunset_Time_Local']):
    #print(accident_time, sunset_time_local)
    #print(str(int(sunset_time_local.split(':')[0]) + 1))
    start_time = str(int(sunset_time_local.split(':')[0]) - 1) + ':' + sunset_time_local.split(':')[1]
    start_time_no_seconds = datetime.strptime(start_time, '%H:%M').time()
    print(start_time_no_seconds)
    #end_time is same as sunset local time
    end_time = str(int(sunset_time_local.split(':')[0])) + ':' + sunset_time_local.split(':')[1]
    end_time_no_seconds = datetime.strptime(end_time, '%H:%M').time()
    print(end_time_no_seconds)
    accident_time_nosecs = accident_time.split(':')[0] + ':' + accident_time.split(':')[1]
    accident_time_no_seconds = datetime.strptime(accident_time_nosecs, '%H:%M').time()
    print(accident_time_no_seconds)
    print(time_in_range(start_time_no_seconds, end_time_no_seconds, accident_time_no_seconds))
    time_range_status.append(time_in_range(start_time_no_seconds, end_time_no_seconds, accident_time_no_seconds))
print(time_range_status)    

16:58:00
17:58:00
08:21:00
False
16:54:00
17:54:00
17:10:00
True
17:08:00
18:08:00
08:13:00
False
17:07:00
18:07:00
05:42:00
False
17:07:00
18:07:00
06:36:00
False
17:09:00
18:09:00
07:02:00
False
17:06:00
18:06:00
10:19:00
False
17:13:00
18:13:00
16:21:00
False
17:31:00
18:31:00
14:58:00
False
17:28:00
18:28:00
06:43:00
False
17:34:00
18:34:00
15:57:00
False
18:37:00
19:37:00
06:01:00
False
18:40:00
19:40:00
07:38:00
False
19:32:00
20:32:00
10:31:00
False
19:34:00
20:34:00
10:56:00
False
19:34:00
20:34:00
11:06:00
False
19:34:00
20:34:00
11:16:00
False
19:32:00
20:32:00
11:33:00
False
19:33:00
20:33:00
11:50:00
False
19:34:00
20:34:00
13:02:00
False
19:31:00
20:31:00
13:59:00
False
19:34:00
20:34:00
14:53:00
False
19:35:00
20:35:00
15:55:00
False
19:31:00
20:31:00
20:03:00
True
19:33:00
20:33:00
00:29:00
False
19:33:00
20:33:00
10:14:00
False
19:34:00
20:34:00
10:17:00
False
19:35:00
20:35:00
10:53:00
False
19:35:00
20:35:00
11:21:00
False
19:31:00
20:31:00
11:26:00
False
19:35:00
20:

15:49:00
16:49:00
11:42:00
False
15:50:00
16:50:00
13:01:00
False
15:51:00
16:51:00
13:25:00
False
15:51:00
16:51:00
15:58:00
True
15:52:00
16:52:00
16:26:00
True
15:51:00
16:51:00
17:34:00
False
15:50:00
16:50:00
18:22:00
False
15:50:00
16:50:00
18:28:00
False
15:51:00
16:51:00
19:23:00
False
15:49:00
16:49:00
20:03:00
False
15:51:00
16:51:00
20:54:00
False
15:46:00
16:46:00
21:41:00
False
15:42:00
16:42:00
22:06:00
False
15:51:00
16:51:00
01:44:00
False
15:52:00
16:52:00
04:46:00
False
15:52:00
16:52:00
09:31:00
False
15:51:00
16:51:00
13:28:00
False
15:50:00
16:50:00
13:29:00
False
15:48:00
16:48:00
13:22:00
False
15:51:00
16:51:00
15:44:00
False
15:52:00
16:52:00
16:02:00
True
15:52:00
16:52:00
16:17:00
True
15:51:00
16:51:00
16:20:00
True
15:51:00
16:51:00
16:18:00
True
15:48:00
16:48:00
16:21:00
True
15:52:00
16:52:00
16:43:00
True
15:52:00
16:52:00
16:47:00
True
15:52:00
16:52:00
14:27:00
False
15:49:00
16:49:00
19:55:00
False
15:52:00
16:52:00
20:28:00
False
15:49:00
16:49:00
2

10:15:00
False
17:14:00
18:14:00
10:36:00
False
17:15:00
18:15:00
11:48:00
False
17:13:00
18:13:00
12:21:00
False
17:13:00
18:13:00
13:47:00
False
17:13:00
18:13:00
14:49:00
False
17:15:00
18:15:00
15:41:00
False
17:07:00
18:07:00
18:04:00
True
17:07:00
18:07:00
18:32:00
False
17:15:00
18:15:00
18:42:00
False
17:14:00
18:14:00
18:36:00
False
17:15:00
18:15:00
19:23:00
False
17:11:00
18:11:00
18:54:00
False
17:10:00
18:10:00
04:27:00
False
17:11:00
18:11:00
07:26:00
False
17:10:00
18:10:00
08:41:00
False
17:09:00
18:09:00
08:50:00
False
17:08:00
18:08:00
09:15:00
False
17:11:00
18:11:00
13:18:00
False
17:10:00
18:10:00
13:45:00
False
17:11:00
18:11:00
14:01:00
False
17:11:00
18:11:00
14:23:00
False
17:09:00
18:09:00
14:27:00
False
17:11:00
18:11:00
15:19:00
False
17:11:00
18:11:00
15:18:00
False
17:11:00
18:11:00
15:16:00
False
17:11:00
18:11:00
15:24:00
False
17:11:00
18:11:00
15:28:00
False
17:11:00
18:11:00
15:53:00
False
17:11:00
18:11:00
15:54:00
False
17:08:00
18:08:00
16:25:00
Fa

20:05:00
False
18:13:00
19:13:00
20:03:00
False
18:09:00
19:09:00
14:24:00
False
18:12:00
19:12:00
14:48:00
False
18:10:00
19:10:00
16:28:00
False
18:12:00
19:12:00
16:43:00
False
18:08:00
19:08:00
17:31:00
False
18:09:00
19:09:00
20:57:00
False
18:10:00
19:10:00
06:47:00
False
18:11:00
19:11:00
16:06:00
False
18:08:00
19:08:00
17:08:00
False
18:09:00
19:09:00
18:28:00
True
18:10:00
19:10:00
22:02:00
False
18:09:00
19:09:00
00:56:00
False
18:07:00
19:07:00
07:41:00
False
18:06:00
19:06:00
08:34:00
False
18:08:00
19:08:00
09:14:00
False
18:06:00
19:06:00
09:12:00
False
18:08:00
19:08:00
09:32:00
False
18:08:00
19:08:00
09:44:00
False
18:06:00
19:06:00
09:54:00
False
18:09:00
19:09:00
10:54:00
False
18:08:00
19:08:00
11:14:00
False
18:07:00
19:07:00
11:31:00
False
18:08:00
19:08:00
12:03:00
False
18:07:00
19:07:00
12:14:00
False
18:08:00
19:08:00
12:33:00
False
18:07:00
19:07:00
13:42:00
False
18:08:00
19:08:00
13:58:00
False
18:04:00
19:04:00
14:05:00
False
18:07:00
19:07:00
14:28:00
Fa

10:38:00
False
19:08:00
20:08:00
11:04:00
False
19:07:00
20:07:00
11:18:00
False
19:07:00
20:07:00
12:34:00
False
19:08:00
20:08:00
14:48:00
False
19:06:00
20:06:00
15:22:00
False
19:07:00
20:07:00
00:22:00
False
19:02:00
20:02:00
09:03:00
False
19:07:00
20:07:00
09:42:00
False
19:03:00
20:03:00
10:06:00
False
19:05:00
20:05:00
11:12:00
False
19:05:00
20:05:00
11:19:00
False
19:07:00
20:07:00
11:34:00
False
19:05:00
20:05:00
11:51:00
False
19:07:00
20:07:00
12:25:00
False
19:07:00
20:07:00
12:37:00
False
19:07:00
20:07:00
12:43:00
False
19:05:00
20:05:00
13:55:00
False
19:07:00
20:07:00
14:49:00
False
19:07:00
20:07:00
15:03:00
False
19:05:00
20:05:00
15:15:00
False
19:07:00
20:07:00
15:30:00
False
19:07:00
20:07:00
16:24:00
False
19:05:00
20:05:00
17:24:00
False
19:04:00
20:04:00
19:58:00
True
19:07:00
20:07:00
20:15:00
False
19:07:00
20:07:00
21:23:00
False
19:04:00
20:04:00
21:20:00
False
19:07:00
20:07:00
22:18:00
False
19:05:00
20:05:00
01:12:00
False
19:06:00
20:06:00
07:16:00
Fa

11:07:00
False
18:27:00
19:27:00
11:48:00
False
18:29:00
19:29:00
12:07:00
False
18:28:00
19:28:00
12:14:00
False
18:28:00
19:28:00
13:14:00
False
18:29:00
19:29:00
14:58:00
False
18:28:00
19:28:00
18:02:00
False
18:30:00
19:30:00
01:52:00
False
18:27:00
19:27:00
07:24:00
False
18:27:00
19:27:00
07:24:00
False
18:30:00
19:30:00
08:30:00
False
18:27:00
19:27:00
08:18:00
False
18:31:00
19:31:00
08:44:00
False
18:30:00
19:30:00
08:39:00
False
18:29:00
19:29:00
09:48:00
False
18:31:00
19:31:00
09:58:00
False
18:29:00
19:29:00
10:40:00
False
18:28:00
19:28:00
11:38:00
False
18:28:00
19:28:00
11:46:00
False
18:31:00
19:31:00
11:54:00
False
18:31:00
19:31:00
13:59:00
False
18:29:00
19:29:00
14:19:00
False
18:28:00
19:28:00
14:23:00
False
18:31:00
19:31:00
16:56:00
False
18:31:00
19:31:00
17:54:00
False
18:28:00
19:28:00
20:47:00
False
18:28:00
19:28:00
21:33:00
False
18:30:00
19:30:00
21:38:00
False
18:29:00
19:29:00
22:28:00
False
18:30:00
19:30:00
09:24:00
False
18:29:00
19:29:00
09:58:00
F

20:09:00
20:40:00
False
19:08:00
20:08:00
20:54:00
False
19:06:00
20:06:00
21:52:00
False
19:08:00
20:08:00
22:02:00
False
19:08:00
20:08:00
22:50:00
False
19:05:00
20:05:00
08:00:00
False
19:09:00
20:09:00
08:32:00
False
19:08:00
20:08:00
09:15:00
False
19:08:00
20:08:00
10:32:00
False
19:08:00
20:08:00
10:31:00
False
19:09:00
20:09:00
10:39:00
False
19:08:00
20:08:00
10:32:00
False
19:05:00
20:05:00
11:13:00
False
19:09:00
20:09:00
11:28:00
False
19:09:00
20:09:00
11:43:00
False
19:09:00
20:09:00
11:36:00
False
19:08:00
20:08:00
12:10:00
False
19:10:00
20:10:00
12:47:00
False
19:09:00
20:09:00
12:45:00
False
19:09:00
20:09:00
13:16:00
False
19:07:00
20:07:00
13:48:00
False
19:08:00
20:08:00
13:51:00
False
19:08:00
20:08:00
16:22:00
False
19:10:00
20:10:00
16:52:00
False
19:07:00
20:07:00
18:07:00
False
19:07:00
20:07:00
19:38:00
True
19:10:00
20:10:00
20:14:00
False
19:10:00
20:10:00
20:28:00
False
19:10:00
20:10:00
20:19:00
False
19:09:00
20:09:00
20:33:00
False
19:09:00
20:09:00
21

In [389]:
accidents_data_hwy_westbound_sunset_date_time_df['Accidents_Close_To_Sunset_Status'] = time_range_status
accidents_data_hwy_westbound_sunset_date_time_df.head()

,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,hwy,Direction_West,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Time_UTC_Hours,Sunset_Date_Time,Sunset_Date_Time_Local,Sunset_Time_Local,Accidents_Close_To_Sunset_Status
ID,,,,,,,,,,,,,,,,,,,,,
A-12,1,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,...,True,True,2-8-16,10:58:48 PM,8:21,22:58:48,2-8-16 22:58:48,2016-02-08 17:58:48,17:58:48,False
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,True,True,2-10-16,10:54:32 PM,17:10,22:54:32,2-10-16 22:54:32,2016-02-10 17:54:32,17:54:32,True
A-100,7,99,OH,2,2/11/16 8:13,2/11/16 8:43,39.749916,-84.139359,0.01,Accident on US-35 Westbound at Smithville Rd. ...,...,True,True,2-11-16,11:08:13 PM,8:13,23:08:13,2-11-16 23:08:13,2016-02-11 18:08:13,18:08:13,False
A-143,12,142,OH,3,2/15/16 5:42,2/15/16 6:12,39.975986,-82.996307,0.71,Lane blocked due to accident on I-670 Westboun...,...,True,True,2-15-16,11:07:50 PM,5:42,23:07:50,2-15-16 23:07:50,2016-02-15 18:07:50,18:07:50,False
A-144,13,143,OH,3,2/15/16 6:36,2/15/16 9:00,39.951401,-83.040161,0.01,Left lane closed due to accident on I-70 Westb...,...,True,True,2-15-16,11:07:50 PM,6:36,23:07:50,2-15-16 23:07:50,2016-02-15 18:07:50,18:07:50,False


In [390]:
accidents_data_hwy_westbound_close_to_sunset_df = accidents_data_hwy_westbound_sunset_date_time_df.loc[accidents_data_hwy_westbound_sunset_date_time_df['Accidents_Close_To_Sunset_Status'] == True]
accidents_data_hwy_westbound_close_to_sunset_df.head()


,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,hwy,Direction_West,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Time_UTC_Hours,Sunset_Date_Time,Sunset_Date_Time_Local,Sunset_Time_Local,Accidents_Close_To_Sunset_Status
ID,,,,,,,,,,,,,,,,,,,,,
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,True,True,2-10-16,10:54:32 PM,17:10,22:54:32,2-10-16 22:54:32,2016-02-10 17:54:32,17:54:32,True
A-811,68,810,CA,3,6/21/16 20:03,6/21/16 20:48,37.766987,-121.335495,0.00,Two lanes blocked due to accident on I-205 Wes...,...,True,True,6-21-16,3:31:08 AM,20:03,03:31:08,6-21-16 03:31:08,2016-06-20 20:31:08,20:31:08,True
A-1176,144,1175,CA,3,6/24/16 19:57,6/24/16 20:27,37.701668,-121.769699,0.00,Lane blocked due to accident on I-580 Westboun...,...,True,True,6-24-16,3:32:44 AM,19:57,03:32:44,6-24-16 03:32:44,2016-06-23 20:32:44,20:32:44,True
A-1177,145,1176,CA,3,6/24/16 20:07,6/24/16 21:07,38.710537,-121.306374,0.00,Accident on I-80 Westbound at Exit 100 Antelop...,...,True,True,6-24-16,3:33:56 AM,20:07,03:33:56,6-24-16 03:33:56,2016-06-23 20:33:56,20:33:56,True
A-1591,238,1590,CA,3,6/28/16 20:04,6/28/16 21:04,38.758034,-121.261528,0.00,Right hand shoulder blocked due to accident on...,...,True,True,6-28-16,3:34:12 AM,20:04,03:34:12,6-28-16 03:34:12,2016-06-27 20:34:12,20:34:12,True


In [391]:
len(accidents_data_hwy_westbound_close_to_sunset_df)

182

In [439]:
accidents_data_hwy_westbound_close_to_sunset_df['accident_time_sunset_time'] = list(zip(accidents_data_hwy_westbound_close_to_sunset_df.Accident_Time, accidents_data_hwy_westbound_close_to_sunset_df.Sunset_Time_Local))
accidents_data_hwy_westbound_close_to_sunset_df.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,level_0,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Direction_West,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Time_UTC_Hours,Sunset_Date_Time,Sunset_Date_Time_Local,Sunset_Time_Local,Accidents_Close_To_Sunset_Status,accident_time_sunset_time
ID,,,,,,,,,,,,,,,,,,,,,
A-85,5,84,OH,3,2/10/16 17:10,2/10/16 23:59,41.040714,-81.613144,0.01,Left lane blocked due to accident on I-76 West...,...,True,2-10-16,10:54:32 PM,17:10,22:54:32,2-10-16 22:54:32,2016-02-10 17:54:32,17:54:32,True,"(17:10, 17:54:32)"
A-811,68,810,CA,3,6/21/16 20:03,6/21/16 20:48,37.766987,-121.335495,0.00,Two lanes blocked due to accident on I-205 Wes...,...,True,6-21-16,3:31:08 AM,20:03,03:31:08,6-21-16 03:31:08,2016-06-20 20:31:08,20:31:08,True,"(20:03, 20:31:08)"
A-1176,144,1175,CA,3,6/24/16 19:57,6/24/16 20:27,37.701668,-121.769699,0.00,Lane blocked due to accident on I-580 Westboun...,...,True,6-24-16,3:32:44 AM,19:57,03:32:44,6-24-16 03:32:44,2016-06-23 20:32:44,20:32:44,True,"(19:57, 20:32:44)"
A-1177,145,1176,CA,3,6/24/16 20:07,6/24/16 21:07,38.710537,-121.306374,0.00,Accident on I-80 Westbound at Exit 100 Antelop...,...,True,6-24-16,3:33:56 AM,20:07,03:33:56,6-24-16 03:33:56,2016-06-23 20:33:56,20:33:56,True,"(20:07, 20:33:56)"
A-1591,238,1590,CA,3,6/28/16 20:04,6/28/16 21:04,38.758034,-121.261528,0.00,Right hand shoulder blocked due to accident on...,...,True,6-28-16,3:34:12 AM,20:04,03:34:12,6-28-16 03:34:12,2016-06-27 20:34:12,20:34:12,True,"(20:04, 20:34:12)"


In [453]:
percentage_accident_close_to_sunset = len(accidents_data_hwy_westbound_close_to_sunset_df)/len(accidents_data_hwy_westbound_sunset_df) * 100
print(percentage_accident_close_to_sunset)

4.839138526987504
